<a href="https://colab.research.google.com/github/jlopetegui98/Literary-Fine-Tuning-of-LLM/blob/main/Experiments/experiments_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Experiments with base Mistral 7B model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.2 MB

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers wandb datasets einops gradio sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [37]:
import torch
import simpletransformers
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, wandb, platform, gradio, warnings
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login
import json

In [5]:
# data and models paths
dir_root = './drive/MyDrive/DL-ENS'
dir_data = f'{dir_root}/dataset'
authors_paths = [f'{dir_data}/wilde_complete.txt', f'{dir_data}/kipling_complete.txt']
authors_names = ["Wilde", "Kipling"]
clf_path = f'{dir_root}/models/BertClassifier(BERTAA)_balanced_data.pt'
list_to_generate_path = f'{dir_data}/story_prompts.txt'

In [6]:
# load classifier (wilde vs kipling)
clf = torch.load(clf_path)
clf

In [7]:
# function to read the texts of an specific author
def read_texts(path: str, label, len_to_read =None, max_length = 350):
    text = ''
    with open(path, 'r+', encoding='utf-8') as fd:
      text = fd.read()
      if len_to_read != None:
        text = text[:len_to_read]
    text_splited = text.split()
    dt = {'text': [], 'label': []}
    for i in range(0,len(text_splited),max_length):
      text = ' '.join(text_splited[i:min(i+max_length, len(text_splited))])
      dt['text'].append(text)
      dt['label'].append(label)
    return dt

In [8]:
# iterate through every author and build the dataset
dt = {'text': [], 'label': []}
for i,path in enumerate(authors_paths[1:]):
  dt_i = read_texts(path,i)
  dt['text'].extend(dt_i['text'])
  dt['label'].extend(dt_i['label'])

In [9]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [10]:
# load mistral 7B base model
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [78]:
# def tokenize(tokenizer, text):
#   return tokenizer(f"<s>This are the first lines of a work of fiction. Continue it. [INST] {text} [/INST]", return_tensors = "pt", add_special_tokens = False)

class PromptsDataset(Dataset):
    def __init__(self, sequences, tokenizer):
        self.tokenizer = tokenizer
        self.sequences = self.init_sequences(sequences)

    def init_sequences(self, sequences):
        seqs = []
        for i,seq in enumerate(tqdm(sequences)):
          seqs.append(tokenizer(f"<s>This are the first lines of a work of fiction. Continue it. [INST] {seq} [/INST]", return_tensors = "pt", add_special_tokens = True, max_length = 105, padding='max_length'))
        return seqs

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # print(idx)
        input = self.sequences[idx]
        # print(input.input_ids.shape)
        return input

In [15]:
from tqdm import tqdm

In [83]:
# text = f"<s>This are the first lines of a work of fiction. Continue it. [INST] {dt['text'][5][:100]} [/INST]"

# encodeds = tokenizer(text,return_tensors="pt", add_special_tokens=False)

def clf_exp(model, tokenizer, clf, texts, batch_size = 10):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  dataset = PromptsDataset(texts, tokenizer)
  dataloader = DataLoader(
            dataset, batch_size=batch_size
        )
  label_predictions = []
  for i,input in tqdm(enumerate(dataloader)):
    # tokens = tokenize(tokenizer, text)
    # print(input)
    model_inputs = input.to(device)
    print(model_inputs)
    generated_ids = model.generate(**model_inputs, max_new_tokens=350, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    preds, _ = clf.predict(decoded)
    label_predictions.extend(preds)
    del model_inputs
  return label_predictions

In [27]:
texts = []
with open(list_to_generate_path, 'r+', encoding='utf-8') as fd:
  texts = fd.readlines()
texts

['I tiptoed into the bathroom. If anyone caught me doing this, I’d be in big trouble. I grabbed my mother’s lipstick and brought it back to my bedroom where my brother slept…\n',
 'I peeked through the curtains. There was a limo parked outside with two bodyguards. I heard a knock at the door…\n',
 'I went over to say hello to the cute little baby under the umbrella, but when I reached her, I saw that…\n',
 'The bell rang, and I sprinted toward my locker. I had to get out of there before…\n',
 'I opened the front door to find the UPS man standing on the front stoop, his arm around a cylindrical package that was almost as tall as him. Oh no. Not again, I thought…\n',
 'Irene gripped her mom’s hand harder as they walked through the doors of the imposing gray building. Her mom had promised her they’d never have to come here again, but…\n',
 'The lights dimmed and the curtains opened. I felt like I was going to throw up. Why had I ever thought this was a good idea?…\n',
 'As soon as I board

In [81]:
# predict author for each text in the experimental list
author_preds = clf_exp(model, tokenizer, clf, texts)

100%|██████████| 100/100 [00:00<00:00, 3752.25it/s]
0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
0it [00:00, ?it/s]

{'input_ids': tensor([[[  1,   1, 851,  ...,   2,   2,   2]],

        [[  1,   1, 851,  ...,   2,   2,   2]],

        [[  1,   1, 851,  ...,   2,   2,   2]],

        ...,

        [[  1,   1, 851,  ...,   2,   2,   2]],

        [[  1,   1, 851,  ...,   2,   2,   2]],

        [[  1,   1, 851,  ...,   2,   2,   2]]], device='cuda:0'), 'attention_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 1, 0, 0]]], device='cuda:0')}


ValueError: too many values to unpack (expected 2)